In [1]:
ENV["JULIA_PKG_SERVER"] = ""

using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using DataFrames
using Plots
using CategoricalArrays
using Random
using Imbalance
using MLJBase: machine, transform
using ScientificTypes
using TableTransforms
using CSV
using DataFrames

  Activating project at `~/Documents/GitHub/Imbalance.jl/examples`


### 1. Generate Random Data

In [2]:
rng = Random.default_rng()


num_rows = 100
num_cont_feats = 0
class_probs = [0.5, 0.2, 0.3]

num_vals_per_category = [3, 4, 2, 5]

X, y = generate_imbalanced_data(num_rows, num_cont_feats; class_probs, num_vals_per_category)
X = DataFrame(X)

X = coerce(X, autotype(X, :few_to_finite))

Row,Column1,Column2,Column3,Column4
,Cat…,Cat…,Cat…,Cat…
1,3,4,2,5
2,1,1,1,1
3,3,1,1,3
4,3,3,1,3
5,1,3,2,1
6,3,2,1,1
7,3,2,1,3
8,1,4,2,2
9,3,1,2,5


### 2. Apply SMOTE-N

#### Using Imbalance Only

In [3]:
Xover, yover = smoten(X, y; k = 5, ratios = Dict(0=>1.2, 1=> 1.2, 2=>1.2), rng = 42)

Progress:  67%|███████████████████████████▍             |  ETA: 0:00:00



Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
  class:  1


(186×4 DataFrame
 Row │ Column1  Column2  Column3  Column4 
     │ Cat…     Cat…     Cat…     Cat…    
─────┼────────────────────────────────────
   1 │ 3        4        2        5
   2 │ 1        1        1        1
   3 │ 3        1        1        3
   4 │ 3        3        1        3
   5 │ 1        3        2        1
   6 │ 3        2        1        1
   7 │ 3        2        1        3
   8 │ 1        4        2        2
  ⋮  │    ⋮        ⋮        ⋮        ⋮
 180 │ 3        1        1        5
 181 │ 1        4        1        5
 182 │ 2        1        1        5
 183 │ 3        2        1        5
 184 │ 3        1        1        5
 185 │ 3        1        1        5
 186 │ 1        2        2        4
                          171 rows omitted, CategoricalValue{Int64, UInt32}[2, 0, 0, 1, 0, 0, 0, 2, 0, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

#### Using MLJ

In [4]:
smotenc_model = Imbalance.MLJ.SMOTEN(k=5, ratios=Dict(0=>1.2, 1=> 1.2, 2=>1.2), rng=42)
mach = machine(smotenc_model)
Xover, yover = transform(mach, X, y)
Xover

Row,Column1,Column2,Column3,Column4
,Cat…,Cat…,Cat…,Cat…
1,3,4,2,5
2,1,1,1,1
3,3,1,1,3
4,3,3,1,3
5,1,3,2,1
6,3,2,1,1
7,3,2,1,3
8,1,4,2,2
9,3,1,2,5


#### Using TableTransforms

In [5]:
# For TableTransforms
Xy = CSV.read("./datasets/churn.csv", DataFrame)
y_ind = 17
Xy = coerce(Xy, autotype(Xy, :string_to_multiclass))

types = schema(Xy).scitypes
cat_inds = findall( x -> x <: Finite, types)
Xy = Xy[:, cat_inds]
Xy

Row,Surname,Geography,Gender
,Cat…,Cat…,Cat…
1,Hargrave,France,Female
2,Hill,Spain,Female
3,Onio,France,Female
4,Boni,France,Female
5,Mitchell,Spain,Female
6,Chu,Spain,Male
7,Bartlett,France,Male
8,Obinna,Germany,Female
9,He,France,Male


In [6]:
using Imbalance: TableTransforms.SMOTEN
y_ind = 2
smoten_model_t = SMOTEN(y_ind, ratios=Dict(1=> 1.2, 2=>1.2, 3=>1.2, 4=>1.2), rng=42)
Xyover = Xy |> smoten_model_t

Progress:  67%|███████████████████████████▍             |  ETA: 0:00:00



Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
  class:  2


Row,Surname,Geography,Gender
,Cat…,Cat…,Cat…
1,Hargrave,France,Female
2,Hill,Spain,Female
3,Onio,France,Female
4,Boni,France,Female
5,Mitchell,Spain,Female
6,Chu,Spain,Male
7,Bartlett,France,Male
8,Obinna,Germany,Female
9,He,France,Male
